In [ ]:
# SEE IF ANY PREVIOUS CONCLUSION CHANGED
# PLOT PROMINENCE ANALYSIS

import pandas as pd
import seaborn as sns
import numpy as np
from easyquery import Query
from fast3tree import fast3tree
from dotenv import load_dotenv

load_dotenv()

In [ ]:
VSMDPL_DATA = os.getenv('VSMDPL_DATA') # path to parquet

In [ ]:
hosts_and_subs = pd.read_parquet(VSMDPL_DATA)

<h2>Add corrected positions</h2>

- if x - x_host > box_length/2, shift by -box_length

- if x - x_host < -box_length/2 shift by +box_length

<h4>Add host x, y, z</h4>

In [ ]:

hosts, subs = Query("upid == -1").split(hosts_and_subs)
# subs = pd.merge(subs, hosts, how="left", left_on="upid", right_on="id", suffixes=("", "_host")).sort_values(by='upid')
subs = hosts_and_subs

In [ ]:
box_len = 160

<h2>Make cuts to get SAGA-like data</h2>
<h4>Current cuts</h4>

- vmax_mpeak >= 45 (following vmax_mpeak vs # satellites & using saga # satellites average)

- 3 or more satellites

<h4>Unused cuts</h4>

- mvir >= 1e10

In [ ]:
# Filter by most massive
# subs = df[df['mvir'] >= 1e10]

# Use vmax_mpeak as proxy for luminosity
subs_cut = subs[subs['vmax_mpeak'] >= 52.77].copy()
# subs_cut = subs

subs_cut.size

## Grab Interlopers

In [ ]:
# threshold redshift
dz_thres = 2.75

In [ ]:
hosts = hosts.copy()
hosts["z_vz"] = np.remainder(hosts["z"] + hosts["vz"] / 100.0, box_len)
subs_cut["z_vz"] = np.remainder(subs_cut["z"] + subs_cut["vz"] / 100.0, box_len)

In [ ]:
interloper_groups = pd.DataFrame()
interloper_groups

with fast3tree(np.vstack([subs_cut["x"].values, subs_cut["y"].values]).T) as tree:
    tree.set_boundaries(0, box_len)
    for host in hosts.itertuples(): 
        center = (host.x, host.y)
        idx = tree.query_radius(center, host.rvir * 0.001, periodic=True)
        dz = np.remainder(subs_cut["z_vz"].values[idx] - host.z_vz, box_len)
        idx = idx[(dz < dz_thres) | (dz > box_len - dz_thres)]
        sys_subs = subs_cut.iloc[idx].copy()
        sys_subs["interloper_host_id"] = host.id
        sys_subs["interloper_host_x"] = host.x
        sys_subs["interloper_host_y"] = host.y
        sys_subs["interloper_host_vmax_mpeak"] = host.vmax_mpeak
        interloper_groups = interloper_groups.append(sys_subs)

In [ ]:
# interloper_groups

In [ ]:
len(interloper_groups['interloper_host_id'].unique())

In [ ]:
############### This groups by ID. NO for interlopers)
# Select only satellite systems with 3 or more members
interloper_groups = interloper_groups.groupby(['interloper_host_id']).filter(lambda x: len(x) > 2)
interloper_groups

In [ ]:
average_num_subs = len(interloper_groups)/len(interloper_groups['interloper_host_id'].unique())
average_num_subs

In [ ]:
test_subs_cut = interloper_groups.copy()
test_subs_cut['test_column'] = test_subs_cut[test_subs_cut['vmax_mpeak'] >= 50].groupby('interloper_host_id')['interloper_host_id'].transform('count')
test_subs_cut
average = test_subs_cut[test_subs_cut['test_column'] > 2]['test_column']
average

<h2>Ellipticity calculations</h2>

<h4>Quadrupole moments</h4>

In [ ]:
ellipticity_df = interloper_groups.copy()
ellipticity_df['x_diff_sq'] = (ellipticity_df['x'] - ellipticity_df['interloper_host_x'])**2
ellipticity_df['y_diff_sq'] = (ellipticity_df['y'] - ellipticity_df['interloper_host_y'])**2
ellipticity_df['x_diff_y_diff'] = (ellipticity_df['y'] - ellipticity_df['interloper_host_y'])*(ellipticity_df['x'] - ellipticity_df['interloper_host_x'])
ellipticity_df = ellipticity_df.groupby('interloper_host_id', as_index = False)[['x_diff_sq', 'y_diff_sq', 'x_diff_y_diff']].mean().rename(columns={'x_diff_sq':'Q_xx', 'y_diff_sq':'Q_yy', 'x_diff_y_diff':'Q_xy'})
ellipticity_df


<h4>Ellipticity components</h4>

In [ ]:
def e_1 (Q_xx, Q_yy, Q_xy):
    return (( Q_xx - Q_yy )/(Q_xx + Q_yy + 2*(Q_xx*Q_yy-Q_xy**2)**(1/2)))
def e_2 (Q_xx, Q_yy, Q_xy):
    return (2*Q_xy/(Q_xx + Q_yy + 2*(Q_xx*Q_yy-Q_xy**2)**(1/2)))
def e_3 (e_1, e_2):
    return ( e_1**2 + e_2**2 )**(1/2)


ellipticity_df['e_1'] = e_1(ellipticity_df['Q_xx'], ellipticity_df['Q_yy'], ellipticity_df['Q_xy'])
ellipticity_df['e_2'] = e_2(ellipticity_df['Q_xx'], ellipticity_df['Q_yy'], ellipticity_df['Q_xy'])
ellipticity_df['e_3'] = e_3(ellipticity_df['e_1'], ellipticity_df['e_2'])

In [ ]:
# sns.set(style="darkgrid", palette="viridis")
custom_style = {
    "figure.facecolor": "212946",
    "axes.facecolor": "212946",
    "savefig.facecolor": "212946", 
    "grid.color": "2A3459",
    "text.color": "0.9",
    "axes.labelcolor": "0.9",
    "xtick.color": "0.9",
    "ytick.color": "0.9",
    "grid.linestyle": "-",
    "lines.solid_capstyle": "round"
}

sns.set_style("darkgrid", custom_style)

In [ ]:
sns.histplot(ellipticity_df["e_3"], bins=7, element='poly', stat="density", fill=False)

In [ ]:
# check about the high ellipticity bois
data = interloper_groups
high_ellipticity_halo_ids = ellipticity_df[ellipticity_df['e_3'] >= 0.9]['interloper_host_id'].unique()
high_ellipticity_data = data[data['upid'].isin(high_ellipticity_halo_ids)]

high_ellipticity_hosts = high_ellipticity_data.groupby(['interloper_host_id'])['id'].count()
average_count_high_ellipticity = high_ellipticity_hosts.mean()
print("Average number of satellites per halo for the high-eccentricity data: {}".format(average_count_high_ellipticity))

all_hosts = data.groupby(['interloper_host_id'])['id'].count()
average_count_all = all_hosts.mean()
print("Average number of satellites per halo for entire data set: {}".format(average_count_all))


<h4>Store ellipticity DataFrame to use in other nbs</h4>

In [ ]:
ellipticity_df_VSMDPL_interlopers = ellipticity_df
%store ellipticity_df_VSMDPL_interlopers

In [ ]:
display(ellipticity_df_VSMDPL_interlopers)

<h2>Highest luminosity analysis</h2>

In [ ]:
# add num satellites
interloper_groups['num_satellites'] = interloper_groups.groupby('interloper_host_id')['interloper_host_id'].transform('count')
highest_luminosity_subs = interloper_groups.groupby('interloper_host_id', as_index = False)[['Mr', 'num_satellites']].min()

In [ ]:
highest_luminosity_ellipticity = pd.merge(highest_luminosity_subs, ellipticity_df, how="left", left_on="interloper_host_id", right_on="interloper_host_id")[['interloper_host_id', 'Mr', 'e_3']]

In [ ]:
# get rid of hosts which were filtered out by 
highest_luminosity_ellipticity_VSMDPL_interlopers = highest_luminosity_ellipticity[~highest_luminosity_ellipticity['e_3'].isna()]
%store highest_luminosity_ellipticity_VSMDPL_interlopers
highest_luminosity_ellipticity_VSMDPL_interlopers

<h2>Concat highest luminosity w/ log(vmax_mpeak)</h2>

In [ ]:
highest_vmax_subs = interloper_groups.groupby('interloper_host_id', as_index = False)['vmax_mpeak'].max()

In [ ]:
highest_vmax_ellipticity = pd.merge(highest_vmax_subs, ellipticity_df, how="left", left_on="interloper_host_id", right_on="interloper_host_id")[['interloper_host_id', 'vmax_mpeak', 'e_3']]
highest_vmax_ellipticity

In [ ]:
highest_vmax_ellipticity['log_vmax_mpeak'] = np.log(highest_vmax_ellipticity['vmax_mpeak'])

In [ ]:
highest_vmax_ellipticity_VSMDPL_interlopers = highest_vmax_ellipticity[~highest_vmax_ellipticity['e_3'].isna()]
%store highest_vmax_ellipticity_VSMDPL_interlopers
highest_vmax_ellipticity_VSMDPL_interlopers

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax1 = plt.subplots()
ax1.set_xlabel('ellipticity (2D)')
ax1.set_ylabel('log(vmax_mpeak)')
sp1 = sns.scatterplot(data=highest_vmax_ellipticity_VSMDPL_interlopers, x='e_3', y='log_vmax_mpeak', ax=ax1, s=2)

In [ ]:
fig = sp1.figure.get_figure()
fig.set_size_inches([6, 4])
fig.savefig("SimVmax_mpeakVsEllipticity.png",dpi=600)

<h2>Average # satellites per vmax_mpeak</h2>

In [ ]:
hosts_num_vmax = interloper_groups.groupby('interloper_host_id').first()[['interloper_host_vmax_mpeak', 'num_satellites']]
hosts_num_vmax

Bin hosts by vmax_mpeak

In [ ]:
hosts_num_vmax_averages = hosts_num_vmax.groupby(pd.cut(hosts_num_vmax['interloper_host_vmax_mpeak'], np.linspace(135, 340, 20))).mean()
hosts_num_vmax_averages

In [ ]:
fig2, ax2 = plt.subplots()
ax2.set_xlabel('vmax_mpeak')
ax2.set_ylabel('average # satellites')
sp2 = sns.lineplot(hosts_num_vmax_averages['interloper_host_vmax_mpeak'], hosts_num_vmax_averages['num_satellites'], ax=ax2)

In [ ]:
fig2 = sp2.figure.get_figure()
fig2.set_size_inches([6, 4])
fig2.savefig("NumsubsVsVmax_mpeak.png",dpi=600)

In [ ]:
fig3, ax3 = plt.subplots()
ax3.set_xlabel('vmax_mpeak')
ax3.set_ylabel('# satellites')
sp3 = sns.scatterplot(hosts_num_vmax['vmax_mpeak_host'], hosts_num_vmax['num_satellites'], ax=ax3, s=2)

In [ ]:
import scipy as sp
spearmanr_results = sp.stats.spearmanr(hosts_num_vmax['vmax_mpeak_host'], hosts_num_vmax['num_satellites'], axis=0, nan_policy='propagate', alternative='two-sided')
spearmanr_results

In [ ]:
fig3 = sp3.figure.get_figure()
fig3.set_size_inches([6, 4])
fig3.savefig("NumsubsVsVmax_mpeak_scatter.png",dpi=600)